#### **Note.** This Jupyter notebook is designed to work standalone without any additional file. The only exception is that after you clone the github repo, you need to manually upload your pretrained models (e.g., models trained with normal or exponential data as described in README) to respective folders.

#### This notebook seems to work fine on colab.research.google.com. On your local computer, some libraries might need to be installed beforehand.

In [0]:
#clone the repository to obtain pretained models and drawing utilities
#install required packages that are not automatically installed on Colab
!git clone https://github.com/wouterkool/attention-learn-to-route.git
!pip install tensorboard_logger
%cd /content/attention-learn-to-route


Cloning into 'attention-learn-to-route'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 246 (delta 0), reused 1 (delta 0), pack-reused 243
Receiving objects: 100% (246/246), 74.66 MiB | 47.05 MiB/s, done.
Resolving deltas: 100% (54/54), done.
/content/attention-learn-to-route


In [0]:
!python generate_data.py --problem tsp --graph_size 50 --name uni --seed 1234


[[0.1915194503788923, 0.6221087710398319], [0.4377277390071145, 0.7853585837137692], [0.7799758081188035, 0.2725926052826416], [0.2764642551430967, 0.8018721775350193], [0.9581393536837052, 0.8759326347420947], [0.35781726995786667, 0.5009951255234587], [0.6834629351721363, 0.7127020269829002], [0.37025075479039493, 0.5611961860656249], [0.5030831653078097, 0.013768449590682241], [0.772826621612374, 0.8826411906361166], [0.3648859839013723, 0.6153961784334937], [0.07538124164297655, 0.3688240060019745], [0.9331401019825216, 0.6513781432265774], [0.3972025777261542, 0.7887301429407455], [0.31683612216887125, 0.5680986526260692], [0.8691273895612258, 0.43617342389567937], [0.8021476420801591, 0.14376682451456457], [0.7042609711183354, 0.7045813081895725], [0.21879210567408858, 0.924867628615565], [0.44214075540417663, 0.9093159589724725], [0.0598092227798519, 0.18428708381381365], [0.04735527880151513, 0.6748809435823302], [0.5946247799344488, 0.5333101629987506], [0.04332406269480349, 0

In [0]:
%%writefile generate_data.py

import argparse
import os
import numpy as np
from utils.data_utils import check_extension, save_dataset


def generate_tsp_data(dataset_size, tsp_size):
    l = []
    for i in range(dataset_size):
        a = np.random.normal(0.5, 0.25, size=(1, tsp_size, 2))
        amin = np.min(a, axis=1)
        amax = np.max(a, axis=1)
        r = (a - amin) / (amax - amin)
        l.append(r.tolist()[0])
    return l


    #return np.random.uniform(size=(dataset_size, tsp_size, 2)).tolist()


def generate_vrp_data(dataset_size, vrp_size):
    CAPACITIES = {
        10: 20.,
        20: 30.,
        50: 40.,
        100: 50.
    }
    return list(zip(
        np.random.uniform(size=(dataset_size, 2)).tolist(),  # Depot location
        np.random.uniform(size=(dataset_size, vrp_size, 2)).tolist(),  # Node locations
        np.random.randint(1, 10, size=(dataset_size, vrp_size)).tolist(),  # Demand, uniform integer 1 ... 9
        np.full(dataset_size, CAPACITIES[vrp_size]).tolist()  # Capacity, same for whole dataset
    ))


def generate_op_data(dataset_size, op_size, prize_type='const'):
    depot = np.random.uniform(size=(dataset_size, 2))
    loc = np.random.uniform(size=(dataset_size, op_size, 2))

    # Methods taken from Fischetti et al. 1998
    if prize_type == 'const':
        prize = np.ones((dataset_size, op_size))
    elif prize_type == 'unif':
        prize = (1 + np.random.randint(0, 100, size=(dataset_size, op_size))) / 100.
    else:  # Based on distance to depot
        assert prize_type == 'dist'
        prize_ = np.linalg.norm(depot[:, None, :] - loc, axis=-1)
        prize = (1 + (prize_ / prize_.max(axis=-1, keepdims=True) * 99).astype(int)) / 100.

    # Max length is approximately half of optimal TSP tour, such that half (a bit more) of the nodes can be visited
    # which is maximally difficult as this has the largest number of possibilities
    MAX_LENGTHS = {
        20: 2.,
        50: 3.,
        100: 4.
    }

    return list(zip(
        depot.tolist(),
        loc.tolist(),
        prize.tolist(),
        np.full(dataset_size, MAX_LENGTHS[op_size]).tolist()  # Capacity, same for whole dataset
    ))


def generate_pctsp_data(dataset_size, pctsp_size, penalty_factor=3):
    depot = np.random.uniform(size=(dataset_size, 2))
    loc = np.random.uniform(size=(dataset_size, pctsp_size, 2))

    # For the penalty to make sense it should be not too large (in which case all nodes will be visited) nor too small
    # so we want the objective term to be approximately equal to the length of the tour, which we estimate with half
    # of the nodes by half of the tour length (which is very rough but similar to op)
    # This means that the sum of penalties for all nodes will be approximately equal to the tour length (on average)
    # The expected total (uniform) penalty of half of the nodes (since approx half will be visited by the constraint)
    # is (n / 2) / 2 = n / 4 so divide by this means multiply by 4 / n,
    # However instead of 4 we use penalty_factor (3 works well) so we can make them larger or smaller
    MAX_LENGTHS = {
        20: 2.,
        50: 3.,
        100: 4.
    }
    penalty_max = MAX_LENGTHS[pctsp_size] * (penalty_factor) / float(pctsp_size)
    penalty = np.random.uniform(size=(dataset_size, pctsp_size)) * penalty_max

    # Take uniform prizes
    # Now expectation is 0.5 so expected total prize is n / 2, we want to force to visit approximately half of the nodes
    # so the constraint will be that total prize >= (n / 2) / 2 = n / 4
    # equivalently, we divide all prizes by n / 4 and the total prize should be >= 1
    deterministic_prize = np.random.uniform(size=(dataset_size, pctsp_size)) * 4 / float(pctsp_size)

    # In the deterministic setting, the stochastic_prize is not used and the deterministic prize is known
    # In the stochastic setting, the deterministic prize is the expected prize and is known up front but the
    # stochastic prize is only revealed once the node is visited
    # Stochastic prize is between (0, 2 * expected_prize) such that E(stochastic prize) = E(deterministic_prize)
    stochastic_prize = np.random.uniform(size=(dataset_size, pctsp_size)) * deterministic_prize * 2

    return list(zip(
        depot.tolist(),
        loc.tolist(),
        penalty.tolist(),
        deterministic_prize.tolist(),
        stochastic_prize.tolist()
    ))


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--filename", help="Filename of the dataset to create (ignores datadir)")
    parser.add_argument("--data_dir", default='data', help="Create datasets in data_dir/problem (default 'data')")
    parser.add_argument("--name", type=str, required=True, help="Name to identify dataset")
    parser.add_argument("--problem", type=str, default='all',
                        help="Problem, 'tsp', 'vrp', 'pctsp' or 'op_const', 'op_unif' or 'op_dist'"
                             " or 'all' to generate all")
    parser.add_argument('--data_distribution', type=str, default='all',
                        help="Distributions to generate for problem, default 'all'.")

    parser.add_argument("--dataset_size", type=int, default=10000, help="Size of the dataset")
    parser.add_argument('--graph_sizes', type=int, nargs='+', default=[20, 50, 100],
                        help="Sizes of problem instances (default 20, 50, 100)")
    parser.add_argument("-f", action='store_true', help="Set true to overwrite")
    parser.add_argument('--seed', type=int, default=1234, help="Random seed")

    opts = parser.parse_args()

    assert opts.filename is None or (len(opts.problems) == 1 and len(opts.graph_sizes) == 1), \
        "Can only specify filename when generating a single dataset"

    distributions_per_problem = {
        'tsp': [None],
        'vrp': [None],
        'pctsp': [None],
        'op': ['const', 'unif', 'dist']
    }
    if opts.problem == 'all':
        problems = distributions_per_problem
    else:
        problems = {
            opts.problem:
                distributions_per_problem[opts.problem]
                if opts.data_distribution == 'all'
                else [opts.data_distribution]
        }

    for problem, distributions in problems.items():
        for distribution in distributions or [None]:
            for graph_size in opts.graph_sizes:

                datadir = os.path.join(opts.data_dir, problem)
                os.makedirs(datadir, exist_ok=True)

                if opts.filename is None:
                    filename = os.path.join(datadir, "{}{}{}_{}_seed{}.pkl".format(
                        problem,
                        "_{}".format(distribution) if distribution is not None else "",
                        graph_size, opts.name, opts.seed))
                else:
                    filename = check_extension(opts.filename)

                assert opts.f or not os.path.isfile(check_extension(filename)), \
                    "File already exists! Try running with -f option to overwrite."

                np.random.seed(opts.seed)
                if problem == 'tsp':
                    dataset = generate_tsp_data(opts.dataset_size, graph_size)
                elif problem == 'vrp':
                    dataset = generate_vrp_data(
                        opts.dataset_size, graph_size)
                elif problem == 'pctsp':
                    dataset = generate_pctsp_data(opts.dataset_size, graph_size)
                elif problem == "op":
                    dataset = generate_op_data(opts.dataset_size, graph_size, prize_type=distribution)
                else:
                    assert False, "Unknown problem: {}".format(problem)

                print(dataset[0])

                save_dataset(dataset, filename)

Overwriting generate_data.py


In [0]:
!python generate_data.py --problem tsp --graph_size 50 --name normal --seed 1234


[[0.5457731898028044, 0.3984465631081797], [0.8258169680029086, 0.5459530145800168], [0.19850525349378206, 0.7474509464585226], [0.6588524480476403, 0.49156173911652973], [0.41300448437119913, 0.22182160874569803], [0.7434673945633128, 0.765048307055643], [0.6861601288547691, 0.25900877521594046], [0.31110618817159474, 0.5988158019359355], [0.5265509687019956, 0.647010390278467], [0.7933198734088502, 0.33867139763233695], [0.3493955098439895, 0.4882959881284033], [0.4647804560136093, 0.6914050469644111], [0.7924439606369617, 0.5196445102425945], [0.6052384041875892, 0.293306932311062], [0.35508738070545987, 0.7763042348636506], [0.2925303880639748, 0.6551296106909803], [0.7136189294765549, 0.7741158089854198], [0.6600552988751361, 0.5779558782418842], [0.44476388401625333, 0.5442496056632247], [0.6536337155236683, 1.0], [0.43063066360743674, 0.5033306269432919], [0.41896081547501457, 0.24998652541862812], [0.48062010792093557, 0.44779076270419055], [0.3685797825467948, 0.60153154442474

In [0]:
%%writefile generate_data.py

import argparse
import os
import numpy as np
from utils.data_utils import check_extension, save_dataset


def generate_tsp_data(dataset_size, tsp_size):
    l = []
    for i in range(dataset_size):
        a = np.random.exponential(1, size=(1, tsp_size, 2))
        amin = np.min(a, axis=1)
        amax = np.max(a, axis=1)
        r = (a - amin) / (amax - amin)
        l.append(r.tolist()[0])
    return l


    #return np.random.uniform(size=(dataset_size, tsp_size, 2)).tolist()


def generate_vrp_data(dataset_size, vrp_size):
    CAPACITIES = {
        10: 20.,
        20: 30.,
        50: 40.,
        100: 50.
    }
    return list(zip(
        np.random.uniform(size=(dataset_size, 2)).tolist(),  # Depot location
        np.random.uniform(size=(dataset_size, vrp_size, 2)).tolist(),  # Node locations
        np.random.randint(1, 10, size=(dataset_size, vrp_size)).tolist(),  # Demand, uniform integer 1 ... 9
        np.full(dataset_size, CAPACITIES[vrp_size]).tolist()  # Capacity, same for whole dataset
    ))


def generate_op_data(dataset_size, op_size, prize_type='const'):
    depot = np.random.uniform(size=(dataset_size, 2))
    loc = np.random.uniform(size=(dataset_size, op_size, 2))

    # Methods taken from Fischetti et al. 1998
    if prize_type == 'const':
        prize = np.ones((dataset_size, op_size))
    elif prize_type == 'unif':
        prize = (1 + np.random.randint(0, 100, size=(dataset_size, op_size))) / 100.
    else:  # Based on distance to depot
        assert prize_type == 'dist'
        prize_ = np.linalg.norm(depot[:, None, :] - loc, axis=-1)
        prize = (1 + (prize_ / prize_.max(axis=-1, keepdims=True) * 99).astype(int)) / 100.

    # Max length is approximately half of optimal TSP tour, such that half (a bit more) of the nodes can be visited
    # which is maximally difficult as this has the largest number of possibilities
    MAX_LENGTHS = {
        20: 2.,
        50: 3.,
        100: 4.
    }

    return list(zip(
        depot.tolist(),
        loc.tolist(),
        prize.tolist(),
        np.full(dataset_size, MAX_LENGTHS[op_size]).tolist()  # Capacity, same for whole dataset
    ))


def generate_pctsp_data(dataset_size, pctsp_size, penalty_factor=3):
    depot = np.random.uniform(size=(dataset_size, 2))
    loc = np.random.uniform(size=(dataset_size, pctsp_size, 2))

    # For the penalty to make sense it should be not too large (in which case all nodes will be visited) nor too small
    # so we want the objective term to be approximately equal to the length of the tour, which we estimate with half
    # of the nodes by half of the tour length (which is very rough but similar to op)
    # This means that the sum of penalties for all nodes will be approximately equal to the tour length (on average)
    # The expected total (uniform) penalty of half of the nodes (since approx half will be visited by the constraint)
    # is (n / 2) / 2 = n / 4 so divide by this means multiply by 4 / n,
    # However instead of 4 we use penalty_factor (3 works well) so we can make them larger or smaller
    MAX_LENGTHS = {
        20: 2.,
        50: 3.,
        100: 4.
    }
    penalty_max = MAX_LENGTHS[pctsp_size] * (penalty_factor) / float(pctsp_size)
    penalty = np.random.uniform(size=(dataset_size, pctsp_size)) * penalty_max

    # Take uniform prizes
    # Now expectation is 0.5 so expected total prize is n / 2, we want to force to visit approximately half of the nodes
    # so the constraint will be that total prize >= (n / 2) / 2 = n / 4
    # equivalently, we divide all prizes by n / 4 and the total prize should be >= 1
    deterministic_prize = np.random.uniform(size=(dataset_size, pctsp_size)) * 4 / float(pctsp_size)

    # In the deterministic setting, the stochastic_prize is not used and the deterministic prize is known
    # In the stochastic setting, the deterministic prize is the expected prize and is known up front but the
    # stochastic prize is only revealed once the node is visited
    # Stochastic prize is between (0, 2 * expected_prize) such that E(stochastic prize) = E(deterministic_prize)
    stochastic_prize = np.random.uniform(size=(dataset_size, pctsp_size)) * deterministic_prize * 2

    return list(zip(
        depot.tolist(),
        loc.tolist(),
        penalty.tolist(),
        deterministic_prize.tolist(),
        stochastic_prize.tolist()
    ))


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--filename", help="Filename of the dataset to create (ignores datadir)")
    parser.add_argument("--data_dir", default='data', help="Create datasets in data_dir/problem (default 'data')")
    parser.add_argument("--name", type=str, required=True, help="Name to identify dataset")
    parser.add_argument("--problem", type=str, default='all',
                        help="Problem, 'tsp', 'vrp', 'pctsp' or 'op_const', 'op_unif' or 'op_dist'"
                             " or 'all' to generate all")
    parser.add_argument('--data_distribution', type=str, default='all',
                        help="Distributions to generate for problem, default 'all'.")

    parser.add_argument("--dataset_size", type=int, default=10000, help="Size of the dataset")
    parser.add_argument('--graph_sizes', type=int, nargs='+', default=[20, 50, 100],
                        help="Sizes of problem instances (default 20, 50, 100)")
    parser.add_argument("-f", action='store_true', help="Set true to overwrite")
    parser.add_argument('--seed', type=int, default=1234, help="Random seed")

    opts = parser.parse_args()

    assert opts.filename is None or (len(opts.problems) == 1 and len(opts.graph_sizes) == 1), \
        "Can only specify filename when generating a single dataset"

    distributions_per_problem = {
        'tsp': [None],
        'vrp': [None],
        'pctsp': [None],
        'op': ['const', 'unif', 'dist']
    }
    if opts.problem == 'all':
        problems = distributions_per_problem
    else:
        problems = {
            opts.problem:
                distributions_per_problem[opts.problem]
                if opts.data_distribution == 'all'
                else [opts.data_distribution]
        }

    for problem, distributions in problems.items():
        for distribution in distributions or [None]:
            for graph_size in opts.graph_sizes:

                datadir = os.path.join(opts.data_dir, problem)
                os.makedirs(datadir, exist_ok=True)

                if opts.filename is None:
                    filename = os.path.join(datadir, "{}{}{}_{}_seed{}.pkl".format(
                        problem,
                        "_{}".format(distribution) if distribution is not None else "",
                        graph_size, opts.name, opts.seed))
                else:
                    filename = check_extension(opts.filename)

                assert opts.f or not os.path.isfile(check_extension(filename)), \
                    "File already exists! Try running with -f option to overwrite."

                np.random.seed(opts.seed)
                if problem == 'tsp':
                    dataset = generate_tsp_data(opts.dataset_size, graph_size)
                elif problem == 'vrp':
                    dataset = generate_vrp_data(
                        opts.dataset_size, graph_size)
                elif problem == 'pctsp':
                    dataset = generate_pctsp_data(opts.dataset_size, graph_size)
                elif problem == "op":
                    dataset = generate_op_data(opts.dataset_size, graph_size, prize_type=distribution)
                else:
                    assert False, "Unknown problem: {}".format(problem)

                print(dataset[0])

                save_dataset(dataset, filename)

Overwriting generate_data.py


In [0]:
!python generate_data.py --problem tsp --graph_size 50 --name exp --seed 1234


[[0.0648326217486951, 0.20000194593085394], [0.17892479486847918, 0.31706846852557735], [0.4736812490623627, 0.06446541187677791], [0.09970603374366134, 0.33363725052333143], [0.9949890006177275, 0.4305144740577369], [0.13717763835115243, 0.1424640427612378], [0.35942181891866704, 0.25672793525688753], [0.1433197566207477, 0.16907202541562075], [0.2177430117076497, 0.001464697891078936], [0.4636357954821873, 0.44201932377572994], [0.14065482811304725, 0.19635785733379146], [0.022665070657763353, 0.09383398526421036], [0.8478844476462812, 0.21668707392525388], [0.1570611054629279, 0.3203452326172691], [0.11774335755851065, 0.172353482558476], [0.6368894053753339, 0.11718737423608672], [0.507049774289008, 0.030718709118311188], [0.38077278348636967, 0.25095908060501215], [0.07561298456127859, 0.5343219792063392], [0.18140018772200828, 0.4953858991849524], [0.01741825395965375, 0.04075237052708683], [0.013284204578148438, 0.23113240145815553], [0.2817081802250816, 0.15632047554394604], [0

# BENCHMARK

In [0]:
### At this point, you need to upload trained respective models to folders pretrained/tsp50_normal, pretrained/tsp50_exp

#### Please see README for directions to train models using data from normal and exponential distributions

## model trained by uniform data

In [0]:
### greedy

In [0]:
!python -W ignore eval.py data/tsp/tsp50_uni_seed1234.pkl --model pretrained/tsp_50 --decode_strategy greedy


  [*] Loading model from pretrained/tsp_50/epoch-99.pt
100% 10/10 [00:01<00:00,  6.46it/s]
Average cost: 5.790307998657227 +- 0.00547049343585968
Average serial duration: 0.17067301559448242 +- 0.0013563416490441696
Average parallel duration: 0.00016667286679148674
Calculated total duration: 0:00:01


In [0]:
!python -W ignore eval.py data/tsp/tsp50_normal_seed1234.pkl --model pretrained/tsp_50 --decode_strategy greedy


  [*] Loading model from pretrained/tsp_50/epoch-99.pt
100% 10/10 [00:01<00:00,  6.93it/s]
Average cost: 5.464256763458252 +- 0.006131308078765869
Average serial duration: 0.14702246856689452 +- 0.0001483013366301723
Average parallel duration: 0.00014357662945985793
Calculated total duration: 0:00:01


In [0]:
!python -W ignore eval.py data/tsp/tsp50_exp_seed1234.pkl --model pretrained/tsp_50 --decode_strategy greedy


  [*] Loading model from pretrained/tsp_50/epoch-99.pt
100% 10/10 [00:01<00:00,  6.87it/s]
Average cost: 4.970943450927734 +- 0.00667491614818573
Average serial duration: 0.14891490631103516 +- 0.00015524930235285626
Average parallel duration: 0.00014542471319437027
Calculated total duration: 0:00:01


### sampling

In [0]:
!python -W ignore eval.py data/tsp/tsp50_uni_seed1234.pkl --model pretrained/tsp_50 --decode_strategy sample --width 1280 --eval_batch_size 1


  [*] Loading model from pretrained/tsp_50/epoch-99.pt
Traceback (most recent call last):
  File "eval.py", line 216, in <module>
    eval_dataset(dataset_path, width, opts.softmax_temperature, opts)
  File "eval.py", line 70, in eval_dataset
    results = _eval_dataset(model, dataset, width, softmax_temp, opts, device)
  File "eval.py", line 108, in _eval_dataset
    model.to(device)
  File "/usr/local/lib/python3.6/dist-packages/torch/nn/modules/module.py", line 426, in to
    return self._apply(convert)
  File "/usr/local/lib/python3.6/dist-packages/torch/nn/modules/module.py", line 202, in _apply
    module._apply(fn)
  File "/usr/local/lib/python3.6/dist-packages/torch/nn/modules/module.py", line 224, in _apply
    param_applied = fn(param)
  File "/usr/local/lib/python3.6/dist-packages/torch/nn/modules/module.py", line 424, in convert
    return t.to(device, dtype if t.is_floating_point() else None, non_blocking)
KeyboardInterrupt


KeyboardInterrupt: ignored

In [0]:
!python -W ignore eval.py data/tsp/tsp50_normal_seed1234.pkl --model pretrained/tsp_50 --decode_strategy sample --width 1280 --eval_batch_size 1


In [0]:
!python -W ignore eval.py data/tsp/tsp50_exp_seed1234.pkl --model pretrained/tsp_50 --decode_strategy sample --width 1280 --eval_batch_size 1


### beam 

In [0]:
!python -W ignore eval.py data/tsp/tsp50_uni_seed1234.pkl --model pretrained/tsp_50 --decode_strategy bs --width 1280

  [*] Loading model from pretrained/tsp_50/epoch-99.pt
100% 10/10 [28:33<00:00, 163.04s/it]
Average cost: 5.712504863739014 +- 0.005121081471443177
Average serial duration: 172.1905626197815 +- 0.2216277528338009
Average parallel duration: 0.16815484630838037
Calculated total duration: 0:28:01


In [0]:
!python -W ignore eval.py data/tsp/tsp50_normal_seed1234.pkl --model pretrained/tsp_50 --decode_strategy bs --width 1280


  [*] Loading model from pretrained/tsp_50/epoch-99.pt
100% 10/10 [28:34<00:00, 163.14s/it]
Average cost: 5.355941295623779 +- 0.0058979040384292605
Average serial duration: 172.30156861228943 +- 0.2218609635298895
Average parallel duration: 0.1682632505979389
Calculated total duration: 0:28:02


In [0]:
!python -W ignore eval.py data/tsp/tsp50_exp_seed1234.pkl --model pretrained/tsp_50 --decode_strategy bs --width 1280

  [*] Loading model from pretrained/tsp_50/epoch-99.pt
100% 10/10 [28:33<00:00, 163.09s/it]
Average cost: 4.859969139099121 +- 0.006258894801139832
Average serial duration: 172.24343769416808 +- 0.22165062462430957
Average parallel duration: 0.16820648212321102
Calculated total duration: 0:28:02


## model trained by normal data

### greedy

In [0]:
!python -W ignore eval.py data/tsp/tsp50_uni_seed1234.pkl --model pretrained/tsp50_normal --decode_strategy greedy

  [*] Loading model from pretrained/tsp50_normal/epoch-99.pt
100% 10/10 [00:01<00:00,  6.44it/s]
Average cost: 5.83931827545166 +- 0.005647162795066833
Average serial duration: 0.17155591316223145 +- 0.0013278416777792222
Average parallel duration: 0.00016753507144749166
Calculated total duration: 0:00:01


In [0]:
!python -W ignore eval.py data/tsp/tsp50_normal_seed1234.pkl --model pretrained/tsp50_normal --decode_strategy greedy

  [*] Loading model from pretrained/tsp50_normal/epoch-99.pt
100% 10/10 [00:01<00:00,  6.59it/s]
Average cost: 5.456946849822998 +- 0.006322169899940491
Average serial duration: 0.15520575637817383 +- 0.00018022133770051756
Average parallel duration: 0.00015156812146306038
Calculated total duration: 0:00:01


In [0]:
!python -W ignore eval.py data/tsp/tsp50_exp_seed1234.pkl --model pretrained/tsp50_normal --decode_strategy greedy

  [*] Loading model from pretrained/tsp50_normal/epoch-99.pt
100% 10/10 [00:01<00:00,  6.75it/s]
Average cost: 4.950140953063965 +- 0.006845551133155822
Average serial duration: 0.15149675369262694 +- 0.0001660719194755531
Average parallel duration: 0.000147946048527956
Calculated total duration: 0:00:01


### sampling

In [0]:
!python -W ignore eval.py data/tsp/tsp50_uni_seed1234.pkl --model pretrained/tsp50_normal --decode_strategy sample --width 1280 --eval_batch_size 1


Traceback (most recent call last):
  File "eval.py", line 216, in <module>
  File "eval.py", line 54, in eval_dataset
    model, _ = load_model(opts.model)
  File "/content/attention-learn-to-route/utils/functions.py", line 120, in load_model
    load_data = torch_load_cpu(model_filename)
  File "/content/attention-learn-to-route/utils/functions.py", line 28, in torch_load_cpu
    return torch.load(load_path, map_location=lambda storage, loc: storage)  # Load on CPU
  File "/usr/local/lib/python3.6/dist-packages/torch/serialization.py", line 426, in load
    return _load(f, map_location, pickle_module, **pickle_load_args)
  File "/usr/local/lib/python3.6/dist-packages/torch/serialization.py", line 620, in _load
    deserialized_objects[key]._set_from_file(f, offset, f_should_read_directly)
KeyboardInterrupt


In [0]:
!python -W ignore eval.py data/tsp/tsp50_normal_seed1234.pkl --model pretrained/tsp50_normal --decode_strategy sample --width 1280 --eval_batch_size 1


  [*] Loading model from pretrained/tsp50_normal/epoch-99.pt
^C


In [0]:
!python -W ignore eval.py data/tsp/tsp50_exp_seed1234.pkl --model pretrained/tsp50_normal --decode_strategy sample --width 1280 --eval_batch_size 1


  [*] Loading model from pretrained/tsp50_normal/epoch-99.pt
Traceback (most recent call last):
  File "eval.py", line 216, in <module>
    eval_dataset(dataset_path, width, opts.softmax_temperature, opts)
  File "eval.py", line 69, in eval_dataset
    dataset = model.problem.make_dataset(filename=dataset_path, num_samples=opts.val_size, offset=opts.offset)
  File "/content/attention-learn-to-route/problems/tsp/problem_tsp.py", line 29, in make_dataset
    return TSPDataset(*args, **kwargs)
  File "/content/attention-learn-to-route/problems/tsp/problem_tsp.py", line 65, in __init__
    data = pickle.load(f)
KeyboardInterrupt


### beam search

In [0]:
!python -W ignore eval.py data/tsp/tsp50_uni_seed1234.pkl --model pretrained/tsp50_normal --decode_strategy bs --width 1280

python3: can't open file 'eval.py': [Errno 2] No such file or directory


In [0]:
!python -W ignore eval.py data/tsp/tsp50_normal_seed1234.pkl --model pretrained/tsp50_normal --decode_strategy bs --width 1280

In [0]:
!python -W ignore eval.py data/tsp/tsp50_exp_seed1234.pkl --model pretrained/tsp50_normal --decode_strategy bs --width 1280

## model trained by exp data

### greedy

In [0]:
!python -W ignore eval.py data/tsp/tsp50_uni_seed1234.pkl --model pretrained/tsp50_exp --decode_strategy greedy

  [*] Loading model from pretrained/tsp50_exp/epoch-99.pt
100% 10/10 [00:01<00:00,  6.77it/s]
Average cost: 5.966853141784668 +- 0.008427996635437012
Average serial duration: 0.15056362113952637 +- 0.00016954486007176906
Average parallel duration: 0.00014703478626906872
Calculated total duration: 0:00:01
Traceback (most recent call last):
  File "eval.py", line 216, in <module>
    eval_dataset(dataset_path, width, opts.softmax_temperature, opts)
  File "eval.py", line 99, in eval_dataset
    out_file), "File already exists! Try running with -f option to overwrite."
AssertionError: File already exists! Try running with -f option to overwrite.


In [0]:
!python -W ignore eval.py data/tsp/tsp50_normal_seed1234.pkl --model pretrained/tsp50_exp --decode_strategy greedy

  [*] Loading model from pretrained/tsp50_exp/epoch-99.pt
100% 10/10 [00:01<00:00,  6.77it/s]
Average cost: 5.920501232147217 +- 0.024558398723602295
Average serial duration: 0.1510557460784912 +- 0.000174060727448626
Average parallel duration: 0.00014751537702977657
Calculated total duration: 0:00:01
Traceback (most recent call last):
  File "eval.py", line 216, in <module>
    eval_dataset(dataset_path, width, opts.softmax_temperature, opts)
  File "eval.py", line 99, in eval_dataset
    out_file), "File already exists! Try running with -f option to overwrite."
AssertionError: File already exists! Try running with -f option to overwrite.


In [0]:
!python -W ignore eval.py data/tsp/tsp50_exp_seed1234.pkl --model pretrained/tsp50_exp --decode_strategy greedy

  [*] Loading model from pretrained/tsp50_exp/epoch-99.pt
100% 10/10 [00:01<00:00,  6.88it/s]
Average cost: 4.858183860778809 +- 0.006778181195259094
Average serial duration: 0.14898428916931153 +- 0.00014569734468097037
Average parallel duration: 0.0001454924698919058
Calculated total duration: 0:00:01
Traceback (most recent call last):
  File "eval.py", line 216, in <module>
    eval_dataset(dataset_path, width, opts.softmax_temperature, opts)
  File "eval.py", line 99, in eval_dataset
    out_file), "File already exists! Try running with -f option to overwrite."
AssertionError: File already exists! Try running with -f option to overwrite.


### sampling

In [0]:
!python -W ignore eval.py data/tsp/tsp50_uni_seed1234.pkl --model pretrained/tsp50_exp --decode_strategy sample --width 1280 --eval_batch_size 1

  [*] Loading model from pretrained/tsp50_exp/epoch-99.pt
  3% 332/10000 [00:48<23:44,  6.79it/s]
Exception ignored in: <bound method tqdm.__del__ of   3% 332/10000 [00:48<23:44,  6.79it/s]>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/tqdm/_tqdm.py", line 931, in __del__
    self.close()
  File "/usr/local/lib/python3.6/dist-packages/tqdm/_tqdm.py", line 1133, in close
    self._decr_instances(self)
  File "/usr/local/lib/python3.6/dist-packages/tqdm/_tqdm.py", line 496, in _decr_instances
    cls.monitor.exit()
  File "/usr/local/lib/python3.6/dist-packages/tqdm/_monitor.py", line 52, in exit
    self.join()
  File "/usr/lib/python3.6/threading.py", line 1053, in join
    raise RuntimeError("cannot join current thread")
RuntimeError: cannot join current thread


In [0]:
!python -W ignore eval.py data/tsp/tsp50_normal_seed1234.pkl --model pretrained/tsp50_exp --decode_strategy sample --width 1280 --eval_batch_size 1

  [*] Loading model from pretrained/tsp50_exp/epoch-99.pt
  2% 215/10000 [00:31<23:50,  6.84it/s]Traceback (most recent call last):
  File "eval.py", line 216, in <module>
    eval_dataset(dataset_path, width, opts.softmax_temperature, opts)
  File "eval.py", line 70, in eval_dataset
    results = _eval_dataset(model, dataset, width, softmax_temp, opts, device)
  File "eval.py", line 140, in _eval_dataset
    sequences, costs = model.sample_many(batch, batch_rep=batch_rep, iter_rep=iter_rep)
  File "/content/attention-learn-to-route/nets/attention_model.py", line 290, in sample_many
    batch_rep, iter_rep
  File "/content/attention-learn-to-route/utils/functions.py", line 189, in sample_many
    _log_p, pi = inner_func(input)
  File "/content/attention-learn-to-route/nets/attention_model.py", line 287, in <lambda>
    lambda input: self._inner(*input),  # Need to unpack tuple into arguments
  File "/content/attention-learn-to-route/nets/attention_model.py", line 254, in _inner
    log

In [0]:
!python -W ignore eval.py data/tsp/tsp50_exp_seed1234.pkl --model pretrained/tsp50_exp --decode_strategy sample --width 1280 --eval_batch_size 1

  [*] Loading model from pretrained/tsp50_exp/epoch-99.pt
  3% 259/10000 [00:38<24:27,  6.64it/s]
Exception ignored in: <bound method tqdm.__del__ of   3% 259/10000 [00:38<24:27,  6.64it/s]>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/tqdm/_tqdm.py", line 931, in __del__
    self.close()
  File "/usr/local/lib/python3.6/dist-packages/tqdm/_tqdm.py", line 1133, in close
    self._decr_instances(self)
  File "/usr/local/lib/python3.6/dist-packages/tqdm/_tqdm.py", line 496, in _decr_instances
    cls.monitor.exit()
  File "/usr/local/lib/python3.6/dist-packages/tqdm/_monitor.py", line 52, in exit
    self.join()
  File "/usr/lib/python3.6/threading.py", line 1053, in join
    raise RuntimeError("cannot join current thread")
RuntimeError: cannot join current thread


### beam

In [0]:
!python -W ignore eval.py data/tsp/tsp50_uni_seed1234.pkl --model pretrained/tsp50_exp --decode_strategy bs --width 1280

  [*] Loading model from pretrained/tsp50_exp/epoch-99.pt
100% 10/10 [28:35<00:00, 163.25s/it]
Average cost: 5.777927398681641 +- 0.00553661823272705
Average serial duration: 172.39187142715454 +- 0.22181264541360965
Average parallel duration: 0.1683514369405806
Calculated total duration: 0:28:03


In [0]:
!python -W ignore eval.py data/tsp/tsp50_normal_seed1234.pkl --model pretrained/tsp50_exp --decode_strategy bs --width 1280

  [*] Loading model from pretrained/tsp50_exp/epoch-99.pt
100% 10/10 [28:34<00:00, 163.13s/it]
Average cost: 5.555920124053955 +- 0.011456948518753052
Average serial duration: 172.33502143096925 +- 0.222191301569115
Average parallel duration: 0.1682959193661809
Calculated total duration: 0:28:02


In [0]:
!python -W ignore eval.py data/tsp/tsp50_exp_seed1234.pkl --model pretrained/tsp50_exp --decode_strategy bs --width 1280

  [*] Loading model from pretrained/tsp50_exp/epoch-99.pt
100% 10/10 [28:33<00:00, 163.09s/it]
Average cost: 4.808318614959717 +- 0.006498489379882812
Average serial duration: 172.24097329711915 +- 0.2216681920611858
Average parallel duration: 0.16820407548546792
Calculated total duration: 0:28:02


## baselines

In [0]:
!python -W ignore -m problems.tsp.tsp_baseline farthest_insertion data/tsp/tsp50_uni_seed1234.pkl data/tsp/tsp50_normal_seed1234.pkl data/tsp/tsp50_exp_seed1234.pkl


100% 10000/10000 [00:30<00:00, 332.56it/s]
Average cost: 6.01089711097235 +- 0.0063338420167314705
Average serial duration: 0.005636166501045227 +- 1.371910427608632e-05
Average parallel duration: 0.0028180832505226134
Calculated total duration: 0:00:28
100% 10000/10000 [00:30<00:00, 330.47it/s]
Average cost: 5.62676766145773 +- 0.006968573700586492
Average serial duration: 0.005669829964637756 +- 1.3685034740754971e-05
Average parallel duration: 0.002834914982318878
Calculated total duration: 0:00:28
100% 10000/10000 [00:29<00:00, 333.34it/s]
Average cost: 4.993208009296066 +- 0.007323261600406931
Average serial duration: 0.005620334649085999 +- 1.3236938778876029e-05
Average parallel duration: 0.0028101673245429994
Calculated total duration: 0:00:28


In [0]:
!python -W ignore -m problems.tsp.tsp_baseline nearest_insertion data/tsp/tsp50_uni_seed1234.pkl data/tsp/tsp50_normal_seed1234.pkl data/tsp/tsp50_exp_seed1234.pkl


100% 10000/10000 [00:29<00:00, 333.60it/s]
Average cost: 6.780160105426864 +- 0.007024212373181713
Average serial duration: 0.005618748044967652 +- 1.4629875740859085e-05
Average parallel duration: 0.002809374022483826
Calculated total duration: 0:00:28
100% 10000/10000 [00:30<00:00, 332.50it/s]
Average cost: 6.242503755908468 +- 0.007808897688745703
Average serial duration: 0.005635808920860291 +- 1.5290685357880946e-05
Average parallel duration: 0.0028179044604301454
Calculated total duration: 0:00:28
100% 10000/10000 [00:30<00:00, 331.78it/s]
Average cost: 5.5927249685120755 +- 0.009604986317122565
Average serial duration: 0.005649359655380249 +- 1.6846423591284476e-05
Average parallel duration: 0.0028246798276901244
Calculated total duration: 0:00:28


In [0]:
!python -W ignore -m problems.tsp.tsp_baseline random_insertion data/tsp/tsp50_uni_seed1234.pkl data/tsp/tsp50_normal_seed1234.pkl data/tsp/tsp50_exp_seed1234.pkl


100% 10000/10000 [00:18<00:00, 538.78it/s]
Average cost: 6.131675164274397 +- 0.006716588129048994
Average serial duration: 0.003534122085571289 +- 8.622659002691257e-06
Average parallel duration: 0.0017670610427856445
Calculated total duration: 0:00:17
100% 10000/10000 [00:18<00:00, 536.66it/s]
Average cost: 5.758545970128334 +- 0.007317850449247265
Average serial duration: 0.0035413674116134644 +- 9.960671837706142e-06
Average parallel duration: 0.0017706837058067322
Calculated total duration: 0:00:17
100% 10000/10000 [00:18<00:00, 528.22it/s]
Average cost: 5.111937675153737 +- 0.008077959198099805
Average serial duration: 0.003602067446708679 +- 1.0579160474415033e-05
Average parallel duration: 0.0018010337233543396
Calculated total duration: 0:00:18


In [0]:
!python -W ignore -m problems.tsp.tsp_baseline nn data/tsp/tsp50_uni_seed1234.pkl data/tsp/tsp50_normal_seed1234.pkl data/tsp/tsp50_exp_seed1234.pkl


100% 10/10 [00:05<00:00,  1.96it/s]
Average cost: 7.002710671377182 +- 0.011279827072329818
Average serial duration: 0.465148401260376 +- 0.026978397690219293
Average parallel duration: 0.00046514840126037597
Calculated total duration: 0:00:04
100% 10/10 [00:00<00:00, 16.60it/s]
Average cost: 6.573115315294266 +- 0.010541367987194574
Average serial duration: 0.015581989288330078 +- 2.187372368163587e-06
Average parallel duration: 1.5581989288330077e-05
Calculated total duration: 0:00:00
100% 10/10 [00:00<00:00, 16.61it/s]
Average cost: 5.937869461297989 +- 0.011053396343795159
Average serial duration: 0.015564584732055664 +- 2.044173924094857e-06
Average parallel duration: 1.5564584732055665e-05
Calculated total duration: 0:00:00


In [0]:
!python -W ignore -m problems.tsp.tsp_baseline lkh data/tsp/tsp50_uni_seed1234.pkl data/tsp/tsp50_normal_seed1234.pkl data/tsp/tsp50_exp_seed1234.pkl


/content/attention-learn-to-route/problems/vrp/lkh/LKH-3.0.4 not found, downloading and compiling
--2019-11-08 06:02:49--  http://www.akira.ruc.dk/~keld/research/LKH-3/LKH-3.0.4.tgz
Resolving www.akira.ruc.dk (www.akira.ruc.dk)... 130.225.220.151
Connecting to www.akira.ruc.dk (www.akira.ruc.dk)|130.225.220.151|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://akira.ruc.dk/~keld/research/LKH-3/LKH-3.0.4.tgz [following]
--2019-11-08 06:02:50--  http://akira.ruc.dk/~keld/research/LKH-3/LKH-3.0.4.tgz
Resolving akira.ruc.dk (akira.ruc.dk)... 130.225.220.230
Connecting to akira.ruc.dk (akira.ruc.dk)|130.225.220.230|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2308964 (2.2M) [application/x-gzip]
Saving to: ‘LKH-3.0.4.tgz’

LKH-3.0.4.tgz       100%[===================>]   2.20M   466KB/s    in 6.2s    

2019-11-08 06:02:56 (366 KB/s) - ‘LKH-3.0.4.tgz’ saved [2308964/2308964]

LKH-3.0.4/
LKH-3.0.4/pr2392.par
LKH-3.0.4/whizzkids96.a